# Obtaining data

In this notebook, we're going to load a few different datasets, inspect them, and do any necessary cleaning so that we have datasets that we can measure the effectiveness of KNN Imputation against.

Our datasets will include:

* The Iris Dataset
* The Titanic Dataset 
* The King County Housing Dataset
* The Terry Stops Dataset
* 

Let's load our necessary libraries.

# Importing Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Inspecting and Scrubbing

## Iris Dataset

The Iris Dataset is a perfectly clean clean and curated dataset.  The features are all continuous variables while the target is divided into 3 separate classes.  The data is evenly balanced and the target variable even comes out of the box with numerical dummy labels.  We're going to take a step back and relabel the target classes for future interpretability.

In [2]:
from sklearn import datasets

# loading dataset
data = datasets.load_iris()

# Concatenating data into Dataframe
# Using Numpy's Concatenate function to join (np.c_) to join the datasets 
iris_df = pd.DataFrame(data=np.c_[data['data'], data['target']], 
                                  columns=data['feature_names'] + ['target'])
iris_df['target'] = iris_df.target.astype(int)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [3]:
# Returning labels to classes
class_ = ['0', '1', '2']
label = ['setosa', 'versicolor', 'virginica']
iris_df['target'] = iris_df.target.astype(str)
for i in range(len(iris_df)):
    for j in range(len(class_)):
        if iris_df['target'][i] == '0':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                'setosa')
        elif iris_df['target'][i] == '1':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                label[j])
        elif iris_df['target'][i] == '2':
            iris_df['target'][i] = iris_df['target'][i].replace(class_[j], 
                                                                label[j])
iris_df.target.value_counts()
            

versicolor    50
setosa        50
virginica     50
Name: target, dtype: int64

In [4]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [5]:
# exporting to csv
iris_df.to_csv('datasets/iris/iris_cleaned')

## Titanic Dataset
This is a classic which again is used for classification problems.  This will be a nice and easy dive into applying KNN Imputation to slightly larger datasets that contain categorical, continuous, and discreet variables.

In [6]:
titanic = pd.read_csv('datasets/titanicdataset/full.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,WikiId,Name_wiki,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,691.0,"Braund, Mr. Owen Harris",22.0,"Bridgerule, Devon, England",Southampton,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,90.0,"Cumings, Mrs. Florence Briggs (née Thayer)",35.0,"New York, New York, US",Cherbourg,"New York, New York, US",4,NaN,1.0
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,865.0,"Heikkinen, Miss Laina",26.0,"Jyväskylä, Finland",Southampton,New York City,14?,NaN,3.0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,127.0,"Futrelle, Mrs. Lily May (née Peel)",35.0,"Scituate, Massachusetts, US",Southampton,"Scituate, Massachusetts, US",D,NaN,1.0
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,627.0,"Allen, Mr. William Henry",35.0,"Birmingham, West Midlands, England",Southampton,New York City,NaN,NaN,3.0


In [7]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  WikiId       1304 non-null   float64
 13  Name_wiki    1304 non-null   object 
 14  Age_wiki     1302 non-null   float64
 15  Hometown     1304 non-null   object 
 16  Boarded      1304 non-null   object 
 17  Destination  1304 non-null   object 
 18  Lifeboat     502 non-null    object 
 19  Body  

Since the target variable is the column `Survived`, we can't use any of the rows where a passenger's survival is unknown **can't be used**.

In [8]:
titanic = titanic[titanic['Survived'].isna() != True]
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  WikiId       889 non-null    float64
 13  Name_wiki    889 non-null    object 
 14  Age_wiki     887 non-null    float64
 15  Hometown     889 non-null    object 
 16  Boarded      889 non-null    object 
 17  Destination  889 non-null    object 
 18  Lifeboat     345 non-null    object 
 19  Body    

### ID Columns

Columns that are used to identify specific people are not useful in classification tasks, as they are not descriptors that can be clustered.  It also will have no bearing on the likelihood of someone surviving the tragedy.  So we'll go ahead and drop these columns as well. We ***will*** keep `PassengerId` for ease of indexing when we start to introduce NaN values in the experiment. 

In [9]:
titanic = titanic.drop(['Name', 'WikiId', 'Name_wiki'], axis=1)
titanic.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Body,Class
0,1,0.0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,22.0,"Bridgerule, Devon, England",Southampton,"Qu'Appelle Valley, Saskatchewan, Canada",NaN,NaN,3.0
1,2,1.0,1,female,38.0,1,0,PC 17599,71.2833,C85,C,35.0,"New York, New York, US",Cherbourg,"New York, New York, US",4,NaN,1.0
2,3,1.0,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,26.0,"Jyväskylä, Finland",Southampton,New York City,14?,NaN,3.0
3,4,1.0,1,female,35.0,1,0,113803,53.1000,C123,S,35.0,"Scituate, Massachusetts, US",Southampton,"Scituate, Massachusetts, US",D,NaN,1.0
4,5,0.0,3,male,35.0,0,0,373450,8.0500,NaN,S,35.0,"Birmingham, West Midlands, England",Southampton,New York City,NaN,NaN,3.0


### Age

The curator of this dataset combined the original "legacy" dataset with data they scraped from Wikipedia.  As a result, we have duplicate columns where Wikipedia was able to confirm or supplement the original data.  Since the column `Age_wiki` has fewer NaN values, we'll keep this one and drop the original `Age` column.

In [10]:
titanic = titanic.drop('Age', axis=1)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        204 non-null    object 
 9   Embarked     889 non-null    object 
 10  Age_wiki     887 non-null    float64
 11  Hometown     889 non-null    object 
 12  Boarded      889 non-null    object 
 13  Destination  889 non-null    object 
 14  Lifeboat     345 non-null    object 
 15  Body         87 non-null     object 
 16  Class        889 non-null    float64
dtypes: float64(4), int64(4), object(9)
memory usage: 125.3+ KB


### Body

The purpose of the `Body` column was to associate Passengers with bodies recovered in the water, with the letters at the end referencing the ship that recovered them.  Even after cleaning it, this column will lead to biased results because of the 100% correlation between a passenger not surviving and their body being found.  Therefore, we'll need to drop this as well. 

In [11]:
titanic = titanic.drop('Body', axis=1)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    float64
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Ticket       891 non-null    object 
 7   Fare         891 non-null    float64
 8   Cabin        204 non-null    object 
 9   Embarked     889 non-null    object 
 10  Age_wiki     887 non-null    float64
 11  Hometown     889 non-null    object 
 12  Boarded      889 non-null    object 
 13  Destination  889 non-null    object 
 14  Lifeboat     345 non-null    object 
 15  Class        889 non-null    float64
dtypes: float64(4), int64(4), object(8)
memory usage: 118.3+ KB


### Ticket
Let's check out the values under `Ticket`, this may just be another form of ID.

In [12]:
titanic.Ticket.value_counts()

CA. 2343      7
1601          7
347082        7
3101295       6
CA 2144       6
             ..
113794        1
C.A. 31026    1
14311         1
248731        1
A4. 54510     1
Name: Ticket, Length: 681, dtype: int64

Let's find out if these repeats are part of a batch.

In [13]:
tickets = titanic[titanic['Ticket'] == '1601']
tickets

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Age_wiki,Hometown,Boarded,Destination,Lifeboat,Class
74,75,1.0,3,male,0,0,1601,56.4958,NaN,S,32.0,"Hong Kong, China",Southampton,New York City,C,3.0
169,170,0.0,3,male,0,0,1601,56.4958,NaN,S,24.0,"Hong Kong, China",Southampton,New York City,NaN,3.0
509,510,1.0,3,male,0,0,1601,56.4958,NaN,S,26.0,"Hong Kong, China",Southampton,New York City,14,3.0
643,644,1.0,3,male,0,0,1601,56.4958,NaN,S,32.0,"Hong Kong, China",Southampton,New York City,C,3.0
692,693,1.0,3,male,0,0,1601,56.4958,NaN,S,37.0,"Hong Kong, China",Southampton,New York City,C,3.0
826,827,0.0,3,male,0,0,1601,56.4958,NaN,S,23.0,"Hong Kong, China",Southampton,New York City,NaN,3.0
838,839,1.0,3,male,0,0,1601,56.4958,NaN,S,32.0,"Hong Kong, China",Southampton,New York City,C,3.0


It appears that this ticket number is attributed to several men from Hong Kong. They are of different age and some survived while others did not.  This leads me to believe that these are not copies.  Perhaps people who were in groups had a better chance of survival!  We'll keep these. 

### Class
The class of the passengers has been doubled.  We'll drop the one with missing values.

In [14]:
titanic = titanic.drop('Class', axis=1)

### Lifeboat

Lifeboat is bound to have a high correlation with survival, but let's see if anyone didn't survive that made it into a lifeboat.  This would create noise that would lessen the effect of collinearity.

In [15]:
lifeboat = titanic[['Survived', 'Lifeboat']]
lifeboat = lifeboat[lifeboat['Lifeboat'].isna() != True]
lifeboat.Survived.value_counts()

1.0    341
0.0      4
Name: Survived, dtype: int64

In [16]:
lifeboat = lifeboat[lifeboat['Survived'] == 0.0]
lifeboat

,Survived,Lifeboat
232,0.0,10
470,0.0,A
605,0.0,A
793,0.0,14


We have 4 people that made it into a lifeboat and did not survive. Let's see how many people survived that weren't in lifeboats 

In [17]:
no_lifeboat = titanic[['Survived', 'Lifeboat']]
no_lifeboat = no_lifeboat[no_lifeboat['Survived'] == 1.0]
no_lifeboat.Lifeboat.unique()

array(['4', '14?', 'D', '15', '?', '13', '8', '14', 'C', '7', '6', '3',
       '16', '12', '10', '11', 'B', '5', 'A', '9', '2', '1', nan],
      dtype=object)

The presence of `NaN` values is a strong indicator that many people who survived were not in lifeboats. This will also help to lessen the effect of multicollinearity. 

Also, because only 4 people died that were in lifeboats and only 2 were in the safe lifeboat, we can probably change these values to represent the **likelihood** of being in a lifeboat.

Our new values will range "Confirmed" (passengers with a lifeboat associated to their ID) to "Highly Likely" (1st class passengers who survived) and "Unlikely" (passengers that either passed away or were 3rd class survivors). 

In [18]:
titanic.Pclass.value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [19]:
for i in range(len(titanic['Lifeboat'])):
    if titanic['Lifeboat'][i] == '?':
        titanic['Lifeboat'][i] = titanic['Lifeboat'][i].replace('?',
                                                                'Unconfirmed')
    elif type(titanic['Lifeboat'][i]) == str:
        titanic['Lifeboat'][i] = 'Confirmed'
    
    elif type(titanic['Lifeboat'][i]) == float:
        
        # First Class survivors
        if (titanic['Pclass'][i] == '1') & (titanic['Survived'][i] == 1.0):
            titanic['Lifeboat'][i] = 'Highly Likely'
            
        # Second Class survivors
        elif (titanic['Pclass'][i] == '2') & (titanic['Survived'][i] == 1.0):
            titanic['Lifeboat'][i] = 'Likely'
            
        # Third Class passengers and passengers who died
        elif (titanic['Pclass'][i] == '3') or (titanic['Survived'][i] == 0.0):
            titanic['Lifeboat'][i] = 'Unlikely'

titanic.Lifeboat.value_counts()            

Unlikely       545
Confirmed      335
Unconfirmed     10
Name: Lifeboat, dtype: int64